In [6]:
from main import play_round
import json

In [7]:
from Dice import Dice
from Board import Board

from Player import Player

In [8]:
p1 = Player("Ale")
p2 = Player("Augusto")

players = [p1, p2]

with open("input_dict0.json") as json_file:
    json_input = json.load(json_file)
    board = Board(json_input["width"], json_input["height"])

    # Transfer input moves to dictionary with integer keys, to get access time in O(1)
    moves = {int(k): int(v) for k, v in json_input["moves"].items()}

    # Play the game
    winner = play_round(players, board, moves)
    print(winner.name)

Ale	 Augusto
16	 0
16	 6
4	 6
4	 10
8	 10
8	 11
9	 11
9	 14
10	 14
10	 20
16	 20
16	 22
32	 22
32	 27
34	 27
34	 33
36	 33
winner is  Ale
Ale
